In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import VGG16, MobileNetV2, InceptionV3, NASNetLarge, NASNetMobile
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from efficientnet import EfficientNetB3, center_crop_and_resize, preprocess_input

from keras import models
from keras import layers
from keras import callbacks

import time

import matplotlib.pyplot as plt

import cv2

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
BASEPATH = "stanford-dogs-dataset/"
BATCH_SIZE = 32
EPOCHS = 100

In [3]:
early_stopping = EarlyStopping(patience=20, verbose=1,restore_best_weights=True, monitor="val_acc")
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=7,verbose=1, monitor="val_acc")

In [7]:
def createModelNASNetMobile(trainable=False, optimizer="adam"):
    base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(512,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(120, activation="softmax"))
    
    if not trainable:
        for layer in base_model.layers:
            layer.trainable = False

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 
  
    return model

In [8]:
model = createModelNASNetMobile()

W0622 10:20:29.653894 4766950848 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


19996672/19993432 [==============================] - 9s 0us/step


# Using the ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255.0,
                                   shear_range=0.1,
                                   rotation_range=10.,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=[0.9, 1.1],
                                   brightness_range=[0.8, 1.2],
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255.0)

validation_datagen = ImageDataGenerator(rescale=1./255.0)

In [10]:
train_generator = train_datagen.flow_from_directory(
    BASEPATH + "train",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    BASEPATH + "test",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    BASEPATH + "val",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 15394 images belonging to 120 classes.
Found 1114 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
481/481 [==============================] - 697s 1s/step - loss: 1.8168 - acc: 0.5603 - val_loss: 0.6825 - val_acc: 0.7916
Epoch 2/100
481/481 [==============================] - 643s 1s/step - loss: 1.2462 - acc: 0.6594 - val_loss: 0.6567 - val_acc: 0.8034
Epoch 3/100
 48/481 [=>............................] - ETA: 8:43 - loss: 1.0620 - acc: 0.6908

KeyboardInterrupt: 

## Evaluation

In [12]:
loss, acc = model.evaluate_generator(test_generator,verbose=0, steps=test_generator.samples // BATCH_SIZE)

In [13]:
print(loss,acc)

0.2808854404836893 0.9389587


In [14]:
model.save("nasnetlarge-model-{:.2f}acc-{:.2f}loss-{:.0f}.hdf5".format(acc, loss, time.time()))